## GAN in Tensorflow

Reference : https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch?hl=ko

이번 시간에는 GAN에서 convolutional layer를 추가해 보자는 아이디어로 발전한 DCGAN에 대해 학습해 보겠습니다!

DCGAN에서는 퍼셉트론을 이용해 층을 쌓는 GAN과 달리 모든 layer를 convolutional layer로 쌓습니다.

DCGAN에서는 pooling layer를 쓰지 않아서 이미지의 크기를 줄일 수가 없기 때문에 stride와 kernal size를 조절해 이미지의 크기를 조절할 수 있습니다.

추가로 batch normalization 과정을 추가해서 학습에 안정성을 부여해 줍니다.

CNN과는 달리 fully connected 과정이 없고 오로지 convolutional layer만을 이용해서 학습을 진행하는 것이 큰 특징입니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Root directory for dataset
dataroot = "/content/drive/MyDrive/new/"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 224

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
latent_dim = 100

# Number of training epochs
num_epochs = 10


In [6]:
## Load CelebA
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dataroot,
  seed=123,
  image_size=(image_size, image_size),
  batch_size=batch_size)

Found 35332 files belonging to 1 classes.


In [7]:
## Preprocessing
augmentation_layer = keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.experimental.preprocessing.CenterCrop(image_size,image_size),
    layers.experimental.preprocessing.Normalization()    
])
image_ds = train_ds.map(lambda x,y: (augmentation_layer(x),y))

In [8]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(224, 224, 3)),
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        # First layer. Input: (h,w,c) = (64,64,3)
        #keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        # Second layer. Input : (32, 32, 64)     
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        # Third layer. Input : (16, 16, 128)     
        layers.Conv2D(256, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),     
        # Forth layer. Input : (8, 8, 256)     
        layers.Conv2D(512, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        # Last layer, Input : (4, 4, 512)
        layers.Conv2D(1, (4, 4), strides=(1, 1)),
        layers.Flatten(),
        layers.Softmax()

    ],
    name="discriminator",
)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 112, 112, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        65600     
                                                                 
 batch_normalization (BatchN  (None, 56, 56, 64)       256       
 ormalization)                                                   
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 56, 56, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        65600     
                                                     

In [9]:
generator = keras.Sequential(
    [   
        
        # First layer, Input: (100)
        layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),     
        layers.Reshape((7, 7, 256)),
        # Second layer, Input: (4, 4, 512)
        layers.Conv2DTranspose(128, (5,5), strides=(1, 1), padding="same"),
        layers.BatchNormalization(),     
        layers.LeakyReLU(),
        # Third layer, Input: (8, 8, 256)
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),     
        layers.LeakyReLU(),
        # Forth layer, Input: (16, 16, 128)
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),     
        layers.LeakyReLU(),
        layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),     
        layers.LeakyReLU(),
        layers.Conv2DTranspose(16, (5, 5), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),     
        layers.LeakyReLU(),
     
        # Final layer, Input: (32, 32, 64)
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding="same",activation = "sigmoid"),
    ],
    name="generator",
)
generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1254400   
                                                                 
 batch_normalization_5 (Batc  (None, 12544)            50176     
 hNormalization)                                                 
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        819328    
 nspose)                                                         
                                                                 
 batch_normalization_6 (Batc  (None, 7, 7, 128)        51

In [10]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = keras.optimizers.Adam(learning_rate=0.0002)
g_optimizer = keras.optimizers.Adam(learning_rate=0.0002)

# Instantiate a loss function.
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

In [11]:
random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))

In [12]:
@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Decode them to fake images
    generated_images = generator(random_latent_vectors)
    # Combine them with real images
    combined_images = tf.concat([generated_images, real_images], axis=0)

    # Assemble labels discriminating real from fake images
    labels = tf.concat(
        [tf.ones((batch_size, 1)), tf.zeros((real_images.shape[0], 1))], axis=0
    )
    # Add random noise to the labels - important trick!
    labels += 0.05 * tf.random.uniform(labels.shape)

    # Train the discriminator
    with tf.GradientTape() as tape:
        predictions = discriminator(combined_images)
        d_loss = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss, discriminator.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Assemble labels that say "all real images"
    misleading_labels = tf.zeros((batch_size, 1))

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    with tf.GradientTape() as tape:
        predictions = discriminator(generator(random_latent_vectors))
        g_loss = loss_fn(misleading_labels, predictions)
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    return d_loss, g_loss, generated_images

In [13]:
img_list = []
G_losses = []
D_losses = []
tf.keras.backend.clear_session()
for epoch in range(num_epochs):
    print("\nStart epoch", epoch + 1)

    for _, (real_images,_) in enumerate(tqdm(image_ds)):
        # Train the discriminator & generator on one batch of real images.
        d_loss, g_loss, generated_images = train_step(real_images)

    # Logging.
    # Print metrics
    print("discriminator loss at epochs %d: %.2f" % (epoch + 1, d_loss))
    print("adversarial loss at epochs %d: %.2f" % (epoch + 1, g_loss))


Start epoch 1


  0%|          | 0/277 [07:12<?, ?it/s]


UnimplementedError: ignored

* 활성화 함수 정리

Discriminator 모델에서 흔히 딥러닝 모델에 쓰는 ReLU 대신 Leaky ReLU 함수를 쓰는 것일까요? 그 이유에 대해 알아보기 전, 자주 쓰이는 활성화 함수 네 가지를 정리해 보겠습니다.

* 시그모이드

output값을 0에서 1 사이로 만들어 주는 비선형 함수입니다.
input값이 너무 크거나 작아지면 기울기가 거의 0이 되는 gradient vanishing 현상이 나타날 수 있습니다.

* Tanh (하이퍼볼릭탄젠트)

시그모이드와 유사하나, -1에서 1 사이의 값을 가집니다. 그렇기 때문에 시그모이드보다 대부분의 경우 학습이 더 잘 이루어집니다.
그러나 시그모이드와 마찬가지로 gradient vanishing 현상이 발생할 수 있습니다.

그렇다면 gradient vanishing 현상이란 무엇일까요?

Gradient Vanishing 현상이란, MLP를 학습시키는 방법인 역전파 (Backpropagation) 중 Gradient항이 사라지는 현상을 말합니다.
이 항이 0이 되거나 0에 가까워져서 학습이 불가능해지는 현상인데요!
이를 해결하기 위해 딥러닝에서는 ReLu 함수를 가장 많이 사용합니다.

* ReLU

ReLU 함수는 x값이 양수면 미분값이 1이 되고, x값이 음수면 미분값이 0이 되는 활성화 함수입니다.
앞의 두 활성화 함수와 달리 대부분의 input 값에 대해 기울기가 0이 되는 것을 방지하는 함수입니다.
그렇기 때문에 학습이 매우 빠른 편입니다. 
하지만 x가 음수일 때 미분값이 0이 되는 현상, 즉 Dead ReLU 현상이 발생할 수 있는데요!

이를 보완한 활성화 함수가 바로 Leaky ReLU 함수입니다.
x가 음수일 때 미분값이 0이 되는 것이 아닌 약간의 기울기값을 갖게 하여 Dead ReLU 현상을 보완할 수 있습니다.